In [ ]:
import nfl_data_py as nfl
import pandas as pd

# Import weekly player stats data from 2009-2023
years = list(range(2009, 2023))
df = nfl.import_weekly_data(years)

df['name'] = df['player_display_name']

# Trim dataset for only WRs, then only taking our relevant feature columns and result columns
wr_df = df.loc[df['position'] == 'WR']

wr_df = wr_df[['player_id', 'name', 'season', 'week', 'receiving_air_yards',
               'receiving_yards_after_catch', 'targets', 'receiving_yards', 'receiving_tds',
               'pacr', 'receptions', 'receiving_first_downs', 'receiving_epa',
               'racr', 'target_share', 'air_yards_share', 'wopr',
               'fantasy_points']]

# Let's keep a list of our features and target variables
features = ['receiving_air_yards', 'receiving_yards_after_catch', 'targets', 'receiving_yards', 'receiving_tds',
            'pacr', 'receptions', 'receiving_first_downs', 'receiving_epa',
            'racr', 'target_share', 'air_yards_share', 'wopr']

target = ['fantasy_points']



In [ ]:
# Aggregate DF by player and season, sort

wr_df_agg = wr_df.groupby(['name', 'season']).sum().reset_index()

wr_df_agg = wr_df_agg.sort_values(by=['name', 'season'])

# Shift the fantasy points to be the target for the next season
# Drop rows where next_season_fantasy_points is NaN (i.e., the last season for each player)

wr_df_agg['next_season_fantasy_points'] = wr_df_agg.groupby('name')['fantasy_points'].shift(-1)

wr_df_agg = wr_df_agg.dropna(subset=['next_season_fantasy_points'])

In [ ]:
from sklearn.model_selection import train_test_split

# Define features and target
features = ['receiving_air_yards', 'receiving_yards_after_catch', 'targets', 'receiving_yards', 'receiving_tds',
            'pacr', 'receptions', 'receiving_first_downs', 'receiving_epa',
            'racr', 'target_share', 'air_yards_share', 'wopr']

X = wr_df_agg[features]
y = wr_df_agg['next_season_fantasy_points']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


In [46]:
# Initialize model, train model
model = RandomForestRegressor(random_state=42)

model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R^2 Score: {r2}")

Mean Absolute Error: 35.162439301255574
Mean Squared Error: 2121.6375427446565
Root Mean Squared Error: 46.06123687814578
R^2 Score: 0.4695763231194695
